# Data Preprocessing for Real-Time Clustering (FIXED - No Data Leakage)

**CRITICAL FIX**: This notebook eliminates data leakage by using only PREVIOUS questions' data.

**What Changed**:
- Cumulative metrics calculated BEFORE processing current question
- Removed `is_correct` from features (that's the target!)
- Renamed features to clarify they're from previous questions

**Expected Results**: Realistic 75-85% accuracy (not 99.71%)

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 2: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Libraries imported successfully")

## Step 3: Load Datasets

In [ ]:
# Load main dataset
df = pd.read_csv('/content/drive/MyDrive/FYP_Data/Merge_Enhanced_Fixed.csv')
print(f"📊 Loaded {len(df)} records")
print(f"   Students: {df['Admission No'].nunique()}")

# Load participant tracking
participant_df = pd.read_csv('/content/drive/MyDrive/FYP_Data/Participant_Tracking.csv')
print(f"\n📊 Loaded {len(participant_df)} participant events")

## Step 4: Filter Participating Students Only

**Why**: Only cluster students who joined the session

In [ ]:
# Get students who joined sessions
participated_students = participant_df[
    participant_df['Event Type'] == 'Joined'
]['Admission No'].unique()

print(f"👥 Students who participated: {len(participated_students)}")

# Filter dataset
df_filtered = df[df['Admission No'].isin(participated_students)].copy()
print(f"✅ Filtered to {len(df_filtered)} records from participating students")

## Step 5: Separate Initial Questions from Regular Questions

In [ ]:
# Separate by Quiz#
df_initial = df_filtered[df_filtered['Quiz#'] == 0].copy()
df_regular = df_filtered[df_filtered['Quiz#'] != 0].copy()

print(f"📋 Initial questions (Quiz# = 0): {len(df_initial)} records")
print(f"   Should be 1 per student: {df_initial['Admission No'].nunique()} students")
print(f"\n📋 Regular questions: {len(df_regular)} records")

## Step 6: **CRITICAL FIX** - Dynamic Cluster Assignment (No Leakage)

**The Fix**: Calculate metrics using ONLY previous questions

In [ ]:
def assign_cluster(prev_accuracy, prev_avg_time, has_network_issue):
    """
    Assign cluster based on PREVIOUS performance only
    
    Parameters:
    - prev_accuracy: Accuracy on Q1 to Q(n-1) [0.0 to 1.0]
    - prev_avg_time: Avg response time on Q1 to Q(n-1) [seconds]
    - has_network_issue: Boolean indicating network problems
    
    Returns:
    - cluster: 'Active', 'Moderate', or 'Passive'
    """
    # Network issue → Passive (can't perform well with bad connection)
    if has_network_issue:
        return 'Passive'
    
    # High accuracy + Fast response → Active
    if prev_accuracy > 0.80 and prev_avg_time < 30:
        return 'Active'
    
    # Medium accuracy + Reasonable response → Moderate
    elif prev_accuracy > 0.50 and prev_avg_time < 60:
        return 'Moderate'
    
    # Everything else → Passive
    else:
        return 'Passive'

print("✅ Cluster assignment function defined (uses previous data only)")

## Step 7: Process Regular Questions (FIXED - No Leakage)

**CRITICAL**: Metrics calculated BEFORE processing current question

In [ ]:
# Sort by student and timestamp (chronological order)
df_regular_sorted = df_regular.sort_values(['Admission No', 'Timestamp']).copy()

# Lists to store processed data
completed_data = []
not_completed_data = []

# Process each student
for student_id in df_regular_sorted['Admission No'].unique():
    student_questions = df_regular_sorted[df_regular_sorted['Admission No'] == student_id]
    
    # Initialize counters (for tracking previous performance)
    correct_count = 0
    total_count = 0
    response_times = []
    
    # Process each question chronologically
    for idx, row in student_questions.iterrows():
        
        # ========== CRITICAL: Calculate from PREVIOUS questions ONLY ==========
        if total_count > 0:
            # Have previous data
            prev_accuracy = correct_count / total_count  # Q1 to Q(n-1) only!
            prev_avg_time = np.mean(response_times)      # Q1 to Q(n-1) only!
        else:
            # First question: use defaults
            prev_accuracy = 0.0
            prev_avg_time = 0.0
        
        # Check network quality
        has_network_issue = (
            row['RTT'] > 1000 or 
            row['Jitter'] > 500 or 
            row['Stability'] < 80
        )
        
        # Assign cluster based on PREVIOUS performance
        cluster = assign_cluster(prev_accuracy, prev_avg_time, has_network_issue)
        
        # Create feature dict (NO is_correct - that's the target!)
        if row['Attempt Status'] == 'Completed':
            features = {
                'student_id': student_id,
                'prev_accuracy': prev_accuracy,           # ✅ From Q1 to Q(n-1)
                'prev_avg_time': prev_avg_time,           # ✅ From Q1 to Q(n-1)
                'total_questions_so_far': total_count,    # ✅ Count before current
                'current_response_time': row['Response Time'],  # ✅ Available
                'cluster': cluster  # Label to predict
            }
            completed_data.append(features)
        else:
            # Not completed: include network params
            features = {
                'student_id': student_id,
                'prev_accuracy': prev_accuracy,
                'prev_avg_time': prev_avg_time,
                'total_questions_so_far': total_count,
                'current_response_time': row['Response Time'],
                'rtt': row['RTT'],
                'jitter': row['Jitter'],
                'stability': row['Stability'],
                'cluster': cluster
            }
            not_completed_data.append(features)
        
        # ========== NOW update counters for NEXT iteration ==========
        if row['Attempt Status'] == 'Completed':
            correct_count += row['Is_Correct']
            total_count += 1
            response_times.append(row['Response Time'])

print(f"✅ Processed {len(completed_data)} completed questions")
print(f"✅ Processed {len(not_completed_data)} not completed questions")
print(f"\n🎯 KEY FIX: Features use ONLY previous questions' data")

## Step 8: Create DataFrames and Check Distribution

In [ ]:
# Convert to DataFrames
df_completed = pd.DataFrame(completed_data)
df_not_completed = pd.DataFrame(not_completed_data)

# Check cluster distribution
print("📊 Cluster Distribution (Completed):")
print(df_completed['cluster'].value_counts())
print(f"\n{df_completed['cluster'].value_counts(normalize=True) * 100}")

# Display sample
print("\n📋 Sample Features (NO is_correct in features!):")
print(df_completed.head())

## Step 9: Prepare Initial Question Features

In [ ]:
# For K-Means initial clustering
X_initial = df_initial[['Response Time', 'RTT', 'Jitter', 'Stability']].values
y_initial = df_initial['Engagement Level'].values

print(f"✅ Initial question features: {X_initial.shape}")
print(f"   Features: Response Time, RTT, Jitter, Stability")

## Step 10: Prepare Completed Question Features (NEW - No Leakage)

In [ ]:
# Features: prev_accuracy, prev_avg_time, total_questions_so_far, current_response_time
# Label: cluster

feature_cols_completed = ['prev_accuracy', 'prev_avg_time', 'total_questions_so_far', 'current_response_time']
X_completed = df_completed[feature_cols_completed].values
y_completed = df_completed['cluster'].values

print(f"✅ Completed question features: {X_completed.shape}")
print(f"   Features: {feature_cols_completed}")
print(f"   ⚠️ NOTE: is_correct REMOVED (that's what we predict!)")
print(f"\n   Labels: {y_completed.shape} clusters to predict")

## Step 11: Prepare Not Completed Features

In [ ]:
if len(df_not_completed) > 0:
    feature_cols_not_completed = [
        'prev_accuracy', 'prev_avg_time', 'total_questions_so_far', 
        'current_response_time', 'rtt', 'jitter', 'stability'
    ]
    X_not_completed = df_not_completed[feature_cols_not_completed].values
    y_not_completed = df_not_completed['cluster'].values
    
    print(f"✅ Not completed features: {X_not_completed.shape}")
    print(f"   Features: {feature_cols_not_completed}")
else:
    X_not_completed = np.array([])
    y_not_completed = np.array([])
    print("ℹ️ No not-completed questions in dataset")

## Step 12: Scale Features

In [ ]:
# Scale initial question features
scaler_initial = StandardScaler()
X_initial_scaled = scaler_initial.fit_transform(X_initial)

# Scale completed question features
scaler_completed = StandardScaler()
X_completed_scaled = scaler_completed.fit_transform(X_completed)

# Scale not completed if exists
if len(X_not_completed) > 0:
    scaler_not_completed = StandardScaler()
    X_not_completed_scaled = scaler_not_completed.fit_transform(X_not_completed)
else:
    scaler_not_completed = None
    X_not_completed_scaled = np.array([])

print("✅ All features scaled (mean=0, std=1)")

## Step 13: Save Preprocessed Data

In [ ]:
import os

# Create output directory
output_dir = '/content/drive/MyDrive/FYP_Data/Preprocessed_Fixed'
os.makedirs(output_dir, exist_ok=True)

# Save everything
np.save(f'{output_dir}/X_initial_scaled.npy', X_initial_scaled)
np.save(f'{output_dir}/y_initial.npy', y_initial)

np.save(f'{output_dir}/X_completed_scaled.npy', X_completed_scaled)
np.save(f'{output_dir}/y_completed.npy', y_completed)

if len(X_not_completed_scaled) > 0:
    np.save(f'{output_dir}/X_not_completed_scaled.npy', X_not_completed_scaled)
    np.save(f'{output_dir}/y_not_completed.npy', y_not_completed)

# Save scalers
with open(f'{output_dir}/scaler_initial.pkl', 'wb') as f:
    pickle.dump(scaler_initial, f)

with open(f'{output_dir}/scaler_completed.pkl', 'wb') as f:
    pickle.dump(scaler_completed, f)

if scaler_not_completed:
    with open(f'{output_dir}/scaler_not_completed.pkl', 'wb') as f:
        pickle.dump(scaler_not_completed, f)

# Save feature names
feature_info = {
    'completed_features': feature_cols_completed,
    'not_completed_features': feature_cols_not_completed if len(df_not_completed) > 0 else [],
    'note': 'is_correct REMOVED to prevent data leakage'
}

with open(f'{output_dir}/feature_names.pkl', 'wb') as f:
    pickle.dump(feature_info, f)

print(f"✅ All files saved to {output_dir}")
print(f"\n📁 Files created:")
print(f"   - X_initial_scaled.npy")
print(f"   - y_initial.npy")
print(f"   - X_completed_scaled.npy")
print(f"   - y_completed.npy")
print(f"   - X_not_completed_scaled.npy (if applicable)")
print(f"   - y_not_completed.npy (if applicable)")
print(f"   - scaler_initial.pkl")
print(f"   - scaler_completed.pkl")
print(f"   - scaler_not_completed.pkl (if applicable)")
print(f"   - feature_names.pkl")
print(f"\n🎯 Data is ready for model training (NO LEAKAGE!)")

## Summary

### What Was Fixed:

1. ✅ **Cumulative metrics** calculated BEFORE processing current question
2. ✅ **Removed is_correct** from features (that's the target!)
3. ✅ **Renamed features** to clarify temporal ordering:
   - `cumulative_accuracy` → `prev_accuracy`
   - `avg_response_time` → `prev_avg_time`
   - `total_questions` → `total_questions_so_far`
4. ✅ **Proper first-question handling** (defaults to 0.0)
5. ✅ **Update counters AFTER** creating training sample

### Expected Results:

- **Accuracy**: 75-85% (realistic, not 99.71%)
- **Model**: Learns engagement patterns, not memorizes answers
- **Production**: Works with real-time constraints

### Next Steps:

1. Run **02_Model_Training_RealTime_Fixed.ipynb** to train models
2. Verify accuracy is in 75-85% range (not 99.71%)
3. Use **03_RealTime_Inference_Demo_Fixed.ipynb** for deployment